In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf,SparkContext

### Boilerplate - Spark Session & HDFS Access

In [2]:
cwd = os.getcwd()
for part in cwd.split('/'):
    if part.lower().startswith('edureka'):
        user_id = part.title()
app_name = '{0} : Logistic Regression'.format(user_id)
app_name

'Edureka_121039 : Logistic Regression'

In [3]:
spark = SparkSession.builder.appName(app_name).getOrCreate()
sparkContext = spark.sparkContext
sqlContext = SQLContext(sparkContext)

In [4]:
def get_hdfs_filepath(file_name):
    my_hdfs = '/user/{0}/datasets'.format(user_id.lower())
    return os.path.join(my_hdfs, file_name)

### Dataset

In [5]:
TITANIC_CSV = get_hdfs_filepath('titanic.csv')

In [9]:
titanic = spark.read.csv(TITANIC_CSV,inferSchema=True,header=True)

In [10]:
titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [11]:
titanic.count(), len(titanic.columns)

(891, 12)

In [12]:
titanic.createTempView("Titanic")

In [13]:
df = spark.sql("select * from titanic")

In [14]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [13]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [14]:
cols=df.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [15]:
final_data = cols.na.drop()

In [16]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [18]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [19]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [20]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],outputCol='features')

In [21]:
from pyspark.ml.classification import LogisticRegression

In [26]:
from pyspark.ml import Pipeline

In [27]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [28]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,gender_encoder,embark_encoder,assembler,log_reg_titanic])

In [29]:
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [30]:
fit_model = pipeline.fit(train_data)

In [31]:
results = fit_model.transform(test_data)

In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [33]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [34]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
+--------+----------+
only showing top 20 rows



In [35]:
AUC = my_eval.evaluate(results)

In [36]:
AUC

0.7621131741821398

In [6]:
spark.stop()